<a href="https://colab.research.google.com/github/diegofuentes51/lab14/blob/main/lab14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Primero, instalo las bibliotecas necesarias para trabajar en Google Colab
!pip install ucimlrepo
!pip install scikit-learn
!pip install pandas

# Ahora importo las librerías necesarias
import pandas as pd
from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import VotingRegressor, RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

# Obtengo el dataset desde el repositorio UCI
productivity_prediction_of_garment_employees = fetch_ucirepo(id=597)

# Convierto los datos a dataframes de pandas
X = pd.DataFrame(productivity_prediction_of_garment_employees.data.features)

# Aseguro que y sea unidimensional extrayendo los valores de la columna correspondiente
y = pd.Series(productivity_prediction_of_garment_employees.data.targets.values.ravel())

# Elimino la columna 'date' y dejo 'actual_productivity' como variable predictora
X = X.drop(columns=['date'])

# Primero, selecciono las columnas numéricas y categóricas
numeric_columns = X.select_dtypes(include=['float64', 'int64']).columns
categorical_columns = X.select_dtypes(include=['object']).columns

# Trato los valores faltantes de las columnas numéricas con la mediana
X[numeric_columns] = X[numeric_columns].fillna(X[numeric_columns].median())

# Trato los valores faltantes de las columnas categóricas con la moda (valor más frecuente)
for col in categorical_columns:
    X[col] = X[col].fillna(X[col].mode()[0])

# Reviso si hay valores faltantes después del tratamiento
print(X.isnull().sum())

# Detecto outliers y los elimino (usando el rango intercuartílico - IQR) SOLO EN COLUMNAS NUMÉRICAS
Q1 = X[numeric_columns].quantile(0.25)
Q3 = X[numeric_columns].quantile(0.75)
IQR = Q3 - Q1

# Elimino los outliers
X = X[~((X[numeric_columns] < (Q1 - 1.5 * IQR)) | (X[numeric_columns] > (Q3 + 1.5 * IQR))).any(axis=1)]
y = y[X.index]

# Si hay variables categóricas, las convierto a variables dummy
X = pd.get_dummies(X)

# Realizo el escalamiento de las variables numéricas
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Separo los datos en entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Defino los modelos que usaré para el ensamblaje
knn = KNeighborsRegressor()
svm = SVR()
lr = LinearRegression()
rf = RandomForestRegressor()
gb = GradientBoostingRegressor()

# Crear el ensamblaje utilizando la técnica de Voting (votación)
voting = VotingRegressor(estimators=[('knn', knn), ('svm', svm), ('lr', lr), ('rf', rf), ('gb', gb)])

# Crear el ensamblaje utilizando la técnica de Stacking (apilamiento)
stacking = StackingRegressor(estimators=[('knn', knn), ('svm', svm), ('rf', rf)], final_estimator=LinearRegression())

# Entreno y hago predicciones con los modelos
voting.fit(X_train, y_train)
stacking.fit(X_train, y_train)

y_pred_voting = voting.predict(X_test)
y_pred_stacking = stacking.predict(X_test)

# Función para evaluar los modelos
def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = mse ** 0.5  # Calculamos la raíz cuadrada del MSE
    print("MSE:", mse)
    print("RMSE:", rmse)
    print("MAE:", mean_absolute_error(y_true, y_pred))
    print("R2:", r2_score(y_true, y_pred))

# Evaluación de los modelos
print("Voting Regressor Metrics:")
evaluate_model(y_test, y_pred_voting)

print("\nStacking Regressor Metrics:")
evaluate_model(y_test, y_pred_stacking)


quarter                  0
department               0
day                      0
team                     0
targeted_productivity    0
smv                      0
wip                      0
over_time                0
incentive                0
idle_time                0
idle_men                 0
no_of_style_change       0
no_of_workers            0
dtype: int64
Voting Regressor Metrics:
MSE: 0.019962369640646578
RMSE: 0.14128825018608793
MAE: 0.09742276579990863
R2: 0.2710682189055258

Stacking Regressor Metrics:
MSE: 0.018623329010727305
RMSE: 0.13646731847122703
MAE: 0.09191712348700724
R2: 0.3199636801606609
